# In preparation ...

just for downloading the some data in place of your data.

In [1]:
%%capture
from kgcnn.data.datasets.QM7Dataset import QM7Dataset
QM7Dataset()

move the data from "~/.kgcnn.datasets.qm7" to your local "qm7" folder. The QM7 data is just some example data for this notebook.

In [2]:
from kgcnn.data.qm import QMDataset

In [3]:
data = QMDataset(
    data_directory="qm7/",
    dataset_name="qm7",
    file_name="qm7.csv",
    file_name_xyz="qm7.xyz"
)

In [4]:
data.prepare_data(
    overwrite=True,
    file_column_name=None,
    make_sdf=True
);

In [5]:
%%capture
from kgcnn.molecule.encoder import OneHotEncoder
data.read_in_memory(
    nodes = [
        'Symbol', 'TotalDegree', 'FormalCharge', 'NumRadicalElectrons', 'Hybridization',
        'IsAromatic', 'IsInRing', 'TotalNumHs', 'CIPCode', "ChiralityPossible", "ChiralTag"
    ],
    encoder_nodes = {
        'Symbol': OneHotEncoder(
            ['B', 'C', 'N', 'O', 'F', 'Si', 'P', 'S', 'Cl', 'As', 'Se', 'Br', 'Te', 'I', 'At'],
            dtype="str"
        ),
        'Hybridization': OneHotEncoder([2, 3, 4, 5, 6]),
        'TotalDegree': OneHotEncoder([0, 1, 2, 3, 4, 5], add_unknown=False),
        'TotalNumHs': OneHotEncoder([0, 1, 2, 3, 4], add_unknown=False),
        'CIPCode': OneHotEncoder(['R', 'S'], add_unknown=False, dtype='str'),
        "ChiralityPossible": OneHotEncoder(["1"], add_unknown=False, dtype='str'),
    },
    edges = ['BondType', 'IsAromatic', 'IsConjugated', 'IsInRing', 'Stereo'],
    encoder_edges = {
        'BondType': OneHotEncoder([1, 2, 3, 12], add_unknown=False),
        'Stereo': OneHotEncoder([0, 1, 2, 3], add_unknown=False)
    },
    graph = None,# ['ExactMolWt', 'NumAtoms'],
    encoder_graph = {},
    
    add_hydrogen=True,
    make_directed=False,
    sanitize=False,
    compute_partial_charges=None,
)

In [6]:
print(data[0])

{'node_symbol': array(['C', 'H', 'H', 'H', 'H'], dtype='<U1'), 'node_number': array([6, 1, 1, 1, 1]), 'node_coordinates': array([[ 0.9983, -0.0025, -0.0044],
       [ 2.0902, -0.0024,  0.0041],
       [ 0.6338,  1.0269,  0.0041],
       [ 0.627 , -0.5277,  0.8781],
       [ 0.6414, -0.5075, -0.9054]]), 'edge_indices': array([[0, 1],
       [0, 2],
       [0, 3],
       [0, 4],
       [1, 0],
       [2, 0],
       [3, 0],
       [4, 0]], dtype=int64), 'edge_number': array([1, 1, 1, 1, 1, 1, 1, 1]), 'node_attributes': array([[0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
        1., 0., 0., 0., 0., 0., 0., 0., 2.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
        0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
        1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
        0., 1., 0., 0., 0., 